In [1]:
names = ["Ling",
"Shang the Monk",
"Kate",
"Marcus",
"Ironclad",
"Hong-Joo",
"Fireguard",
"Kibo",
"Helga",
"Sarge",
"Azuma",
"Midnight",
"Yukka and Shade",
"Jack Bulwark",
"Jet",
"Emperor",
"Lynx",
"Monkey King",
"Cobra"
        ]
cols = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "AA", "AB", "AC", "AD", "AE", "AF", "AG", "AH", "AI", "AJ", "AK", "AL", "AM", "AN", "AO", "AP", "AQ", "AR", "AS", "AT", "AU", "AV", "AW", "AX", "AY", "AZ"]
skip = ["11.0", "12.0", "13.0"]

In [2]:
import xlrd
import xlsxwriter
import pandas as pd

wbRD = xlrd.open_workbook("BalanceSource/Balance1_2_10_7.xlsx")
sheets = wbRD.sheets()

xlsxwriter = pd.ExcelWriter('BalanceResult/BalanceResult1_2_10_7.xlsx', engine='xlsxwriter')
wb = xlsxwriter.book

In [3]:
ws = wb.add_worksheet("Result")

In [4]:
prevPatch = '1.2.10'

In [5]:
def WriteNames(sheet, prows, pcols):
    for i in range(len(names)):
        ws.write(prows + i, pcols, names[i])

def OperateHero(sheet, heroName, startRow, prows, pcols):
    strong = 0
    weak = 0
    for row in reversed(range(1, startRow + 1)):
        if sheet.cell(row, 1).value != heroName or row == 1:
            r = prows + names.index(heroName)
            ws.write(r, pcols + 2, strong)
            ws.write(r, pcols + 3, weak)
            ws.write_formula(r, pcols + 1, '=' + cols[pcols] + str(r + 1) + ' - ' + prevPatch + '!' + cols[pcols] + str(r + 1))
            break             
        winRate = sheet.cell(row, 6).value
        if sheet.cell(row, 3).value >= 100:
            ws.write(prows + names.index(heroName), pcols, winRate)
        if sheet.cell(row, 3).value >= 30:
            if sheet.cell(row, 4).value > 0.56:
                strong = strong + 1
            if sheet.cell(row, 4).value < 0.44:
                weak = weak + 1

def OperateCategory(sheet, catName, startRow, prows, pcols):
    i = 0
    hero = ""
    for row in reversed(range(1, startRow)):
        if sheet.cell(row, 0).value != catName:
            break        
        if hero != sheet.cell(row, 1).value:
            hero = sheet.cell(row, 1).value
            OperateHero(sheet, hero, row, prows, pcols)
            i = i + 1
        
def DoBalanceTable(sheet, padding):
    WriteNames(sheet, padding + 8, 0)
    categories = []
    i = 0
    catName = ""
    for row in reversed(range(1, sheet.nrows)):
        if catName != sheet.cell(row, 0).value:
            categories.append(row)
            catName = sheet.cell(row, 0).value
            if not str(catName) in skip:
                OperateCategory(sheet, catName, row, 8 + padding, 1 + 4*i)
                i = i + 1
    return categories


In [6]:
def WorstMatchups(sheet, catName, prows, pcols):
    data = []
    i = 0
    for row in range(sheet.nrows):
        if catName != sheet.cell(row, 0).value:
            continue
        i = i + 1
        data.append([sheet.cell(row, 1).value, sheet.cell(row, 2).value, sheet.cell(row, 4).value])
    df = pd.DataFrame(data, columns = ['Winner', 'Loser', 'Winrate'])
    df.sort_values(by=['Winrate'], inplace=True)
    xlsxwriter.sheets['Result'] = ws
    df.to_excel(xlsxwriter, sheet_name='Result', startrow=prows, startcol=pcols, index=False)
        

In [7]:
categories1 = DoBalanceTable(sheets[0], 0)
categories2 = DoBalanceTable(sheets[1], len(names) + 10)
categories3 = DoBalanceTable(sheets[2], len(names)*2 + 8 + 10)
categories4 = DoBalanceTable(sheets[3], len(names)*3 + 10 + 8 + 10)

In [8]:
WorstMatchups(sheets[0], sheets[0].cell(categories1[0],0).value, len(names)*4 + 45, 3)
WorstMatchups(sheets[1], sheets[1].cell(categories2[0],0).value, len(names)*4 + 45, 10 + 1)

In [9]:
wb.close()